### Data visualizer to explore data ranges and experiment with data values

![Greedybusiness](/home/jeffreymo572/Kaggles/common/images/Monopoly.jpg)

In [1]:
# Importing
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sn
import holidays

# Dataset
data_dir = "~/Kaggles/data/S3E7/train.csv"
df = pd.read_csv(data_dir)

In [ ]:
# Column info
df.info()

In [ ]:
df.head(10)

In [ ]:
# Renaming columns to be more readable
# Current unknowns: market_segment_type, 
df.columns = ["id", 'num_adults', 'num_children', 'num_weekend_nights', 'num_week_nights', 'meal_plan', 
              'parking', 'room_type', 'lead_time', 'year', 'month', 'date', 'market_segment_type', 'repeated_customer',
              'num_prev_cancellations', 'num_prev_not_cancelled', 'avg_price_per_room', 'num_special_requests',
              'booking_status']

for i, value in enumerate(df.columns.tolist()):
    print(f"{i}: {value}")

#### Possible Notes for cleaning
##### Additions
* Insert ratio of cancellations at index 16 
    * `num_prev_cancellations/(num_prev_not_cancelled+avg_price_per_room)`
* Is weekend
    * `pd.Timestamp(f"{year}-{month:.2f}-{day:.2f}").dayofweek`
    * Note: Monday is 0
* Is holiday
    * Might have to hard code: https://en.wikipedia.org/wiki/Federal_holidays_in_the_United_States
* Total people
    * `num_adults+num_children`
##### Removals
* ID (done)
* date -> weekday & holiday
##### Modifications
* Month/Year/Date into one column?
    * Maybe not since month/year may have seasonal impact on cancellation

##### Questionable
* Parking
* meal_plan

# KNOWN CORRELATIONS
**PEOPLE WHO ORDER MEAL PLAN 3 HAVE A 83% CHANCE TO CANCEL!!!** \
**PEOPLE WITH A BOOKING STATUS OF 0 WILL CANCEL!!!!**

In [ ]:
# Information about data 
# Mean, median, mode, variance, std, etc.
df_info = pd.DataFrame()

df_info['var'] = df.var()
df_info['mean'] = df.mean()
df_info['std'] = df.std()
df_info['var/mean'] = df.var()/df.mean()
df_info['std/mean'] = df.std()/df.mean()

df_info.reset_index(inplace=True)
df_info.columns = ['category', 'var', 'mean', 'std', 'var/mean', 'std/mean']

# Dropping and replacing values
df_info = df_info.drop(0)

df_info

In [ ]:
df['norm_adults'] = df["num_adults"]/max(df['num_adults'])
df

In [ ]:
# Holiday checker
dt = pd.to_datetime(dict(year=df.year, month=df.month, day=df.date), errors='coerce')
df_cleaned = pd.concat([df, dt], axis=1)
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
holiday_range = pd.date_range(start='2017-01-01', end='2020-12-31')
cal = calendar()
holidays = cal.holidays(start=holiday_range.min(), end=holiday_range.max())

df_cleaned.columns = ["id", 'num_adults', 'num_children', 'num_weekend_nights', 'num_week_nights', 'meal_plan', 
              'parking', 'room_type', 'lead_time', 'year', 'month', 'date', 'market_segment_type', 'repeated_customer',
              'num_prev_cancellations', 'num_prev_not_cancelled', 'avg_price_per_room', 'num_special_requests',
              'booking_status', 'y-m-d']

df_cleaned['is_holiday'] = df_cleaned['y-m-d'].isin(holidays)
df_cleaned

In [2]:
from data_cleaner import clean_s3e7

df_cleaned = clean_s3e7(df)

df_cleaned

,num_adults,num_children,num_weekend_nights,num_week_nights,meal_plan,parking,room_type,lead_time,year,month,date,market_segment_type,repeated_customer,num_prev_cancellations,num_prev_not_cancelled,avg_price_per_room,num_special_requests,is_holiday,cancel_ratio,booking_status
0,0.50,0.0,0.000000,0.117647,1,0,0,0.020316,2018,1,14,1,1,0.846154,0.0,0.125000,0.0,0,1.0,0
1,0.50,0.0,0.142857,0.117647,0,0,0,0.264108,2018,7,29,0,0,0.000000,0.0,0.133796,0.0,0,NaN,0
2,0.50,0.0,0.000000,0.058824,0,0,0,0.711061,2018,12,2,0,0,0.000000,0.0,0.096296,0.0,0,NaN,0
3,0.25,0.0,0.000000,0.117647,1,0,0,0.072235,2018,12,1,1,0,0.000000,0.0,0.103704,0.0,0,NaN,0
4,0.50,0.0,0.142857,0.000000,0,0,0,0.582393,2018,10,16,0,0,0.000000,0.0,0.185185,0.0,0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42095,0.75,0.0,0.000000,0.235294,0,0,1,0.361174,2018,12,30,1,0,0.000000,0.0,0.259259,0.4,0,NaN,1
42096,0.50,0.0,0.000000,0.176471,0,0,0,0.076749,2017,9,23,0,0,0.000000,0.0,0.416056,0.0,0,NaN,0
42097,0.50,0.0,0.000000,0.117647,2,0,0,0.659142,2018,7,21,0,0,0.000000,0.0,0.177778,0.0,0,NaN,0
42098,0.25,0.0,0.000000,0.176471,0,0,0,0.011287,2018,11,9,0,0,0.000000,0.0,0.222222,0.0,0,NaN,0


In [3]:
df_cleaned['is_holiday'].value_counts()

0    42100
Name: is_holiday, dtype: int64